## Algorithm

Because of the sparsity of the dataset we need the update our user based collobrative filtering approach to meet the needs of sparse data. This can be acvieved by incorporating a matrix factorization algorithm. Compared regular approaches like collobrative or content filtering *footnote, matrix factorization allows us the uncover the latent features underlying two different factors.This is also the method used by companies such as Netflix (citation).

### Explanation of Matrix Factorization

We want to estimate a matrix $Q_{kxk}$ that captures k latent features for the users and beers. Specifically we want to estimate the vectors $b_i$, $u_j$ of length k for each users and beer in our data sets. The vector $b_i$ for the the beer will represent to which extent the beer i posseses the factor that are being interested by the users, whereas similarly $u_i$ will represent the which extent user j is interested in these factors.

When we take the dot product of these 2 vectors, we can estimate the ratings for a specific userson the specific type of beer. 

\begin{align}
\hat{r_{iu}} & = \ b^T_{i} p_{j}
\end{align}

Assuming our initial rating matrix R has dimensions of $mxn$, we cab combine these vectors into two matrices and illustrate the matrix factorization. 

\begin{align}
\hat{R} & = \ UxB^T
\end{align}

Where U is a $mxk$ matrix that represent each users assosication with latent factors and B is a $nxk$ matrix that represents each beers association with the latent factors. We will use Stochastic Gradient Descent to estimate the matrices Q and P. Our loss/objective function can be defined as :
\begin{align}
\hat{R} & = \ UxB^T
\end{align}



*Explanation

## Data Preperation

In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cosine
from scipy import sparse
import numba


C:\Users\Yamac\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df=pd.read_csv("final_data.csv")
dfc=df[["score_overall","user_id","beer_names","beer_id","brewery_name"]]

##df=pd.read_csv("beer_reviews.csv")
##dfc=df[["review_overall","review_profilename","beer_name","beer_beerid","brewery_name"]]

beers=dfc.groupby('beer_id').count().query("score_overall >=10").index
users=dfc.groupby('user_id').count().query("score_overall >=10").index
df_filtered=dfc[dfc.beer_id.isin(beers)][dfc.user_id.isin(users)]

users=pd.factorize(df_filtered.user_id)[0]
beers=pd.factorize(df_filtered.beer_id)[0]
index_to_userid=dict(zip(users,df_filtered.user_id))
index_into_beerid=dict(zip(beers,df_filtered.beer_id))

index_into_beerid = {v: k for k, v in index_into_beerid.items()}
index_to_userid = {v: k for k, v in index_to_userid.items()}
R=np.zeros((len(index_into_beerid),len(index_to_userid))).T

for index, row in df_filtered.iterrows():
    R[index_to_userid[row['user_id']],index_into_beerid[row['beer_id']]]=row['score_overall']
    
index_into_beerid = {v: k for k, v in index_into_beerid.items()}
index_to_userid = {v: k for k, v in index_to_userid.items()}

index_into_beername=dict(zip(beers, df_filtered.beer_names+ " by "+ df_filtered.brewery_name ))


C:\Users\Yamac\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


## Application Without TensorFlow

In [17]:
def matrix_factor(X,epochs=25000,k=4,lr=0.007,beta=0.001):
    n,m=X.shape
    Q=np.random.rand(n,k)
    P=np.random.rand(m,k).T  
    r_hat=Q@P
    z_i,z_j=np.where(X>0)
    z_i=list(z_i)
    z_j=list(z_j)
    n=len(z_i)
    for epoch in range(epochs):
        if epoch%1000==0:
            loss=np.sum((np.square(X[z_i,z_j]-r_hat[z_i,z_j])))
            print("Epoch : " + str(epoch)+ ", Loss: "+str(loss))
        for u in range(n):
            i=z_i[u]
            j=z_j[u]
            Q[i]=Q[i]+2*lr*(X[i,j]-Q[i]@P[:,j])*P[:,j]-beta*Q[i]
            P[:,j]=P[:,j]+2*lr*(X[i,j]-Q[i]@P[:,j])*Q[i]-beta*P[:,j]
        r_hat=Q@P
    return Q@P
     

    
    

In [5]:
@numba.jit("double[:,:](double[:,:],i8,i8,double,double)",nopython=True)
def matrix_factor_numba(X,epochs=25000,k=4,lr=0.0005,beta=0.0001):
    n,m=X.shape
    Q=np.random.rand(n,k)
    P=np.random.rand(m,k).T  
    z_i,z_j=np.where(X>0)
    z_i=list(z_i)
    z_j=list(z_j)
    n=len(z_i)
    for epoch in range(epochs):
        if epoch%100==0:
            print(epoch)
        for u in range(n):
            i=z_i[u]
            j=z_j[u]    
            Q[i]=Q[i]+2*lr*(X[i,j]-Q[i]@P[:,j])*P[:,j]-beta*Q[i]
            P[:,j]=P[:,j]+2*lr*(X[i,j]-Q[i]@P[:,j])*Q[i]-beta*P[:,j]
    r_hat=Q@P
    return r_hat
     

    

In [67]:
%%time
r_hat1=matrix_factor(X,epochs=10000,k=3,lr=0.005,beta=0.0001)

NameError: name 'matrix_factor' is not defined

In [11]:
%%time
r_hat2=matrix_factor_numba(R,epochs=2500,k=3,lr=0.005,beta=0.0001)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
Wall time: 1h 3min 28s


In [13]:
r_hat=(np.round(r_hat2)*4)/4

MemoryError: 

In [18]:
np.save("r_hat",r_hat2)

In [20]:
r_hat=np.round(r_hat2)

MemoryError: 

In [69]:
np.save("R_hat",r_hat2)

(13562, 19714)

In [70]:
np.save("r_hat",R_f)

## Application With Using Tensorflow

In [ ]:
X=np.array([[1,0,0,0,5],[0,0,4,1,0],[0,2,1,0,0],[0,0,4,4,1],[2,3,1,0,0],[5,3,1,0,0]],dtype="float64")


In [40]:
R_f.shape

(13562, 19714)

In [43]:
    m,n=R_f.shape
    
    Rating= tf.placeholder(tf.float32, [m,n])
    Q = tf.Variable(tf.truncated_normal([int(m), k], stddev=0.2, mean=0), name="users")
    P=tf.Variable(tf.truncated_normal([k, int(n)], stddev=0.2, mean=0), name="beers")
        
    
    r_hat=tf.matmul(Q,P)
    mask =  tf.greater(Rating, 0)
    R_h_values = tf.boolean_mask(r_hat, mask)
    R_values = tf.boolean_mask(Rating, mask)
    
    loss = tf.reduce_mean(tf.squared_difference(R_h_values, R_values))+0.0002*(tf.norm(Q)+tf.norm(P))
    

    optimizer = tf.train.GradientDescentOptimizer(0.001)
    train_step = optimizer.minimize(loss)

c:\users\yamac\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:105: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 267361268 elements. This may consume a large amount of memory.
  num_elements)


In [60]:
tf.reset_default_graph() 

In [50]:
def Matrix_Factorization(X,k=3,lr=0.001,beta=0.0005,epochs=25000,tolerance=0.001):
    m,n=X.shape
    
    Rating= tf.placeholder(tf.float32, [m,n])
    Q = tf.Variable(tf.truncated_normal([int(m), k], stddev=0.2, mean=0), name="users")
    P=tf.Variable(tf.truncated_normal([k, int(n)], stddev=0.2, mean=0), name="beers") 
    
    r_hat=tf.matmul(Q,P)
    mask =  tf.greater(Rating, 0)
    R_h_values = tf.boolean_mask(r_hat, mask)
    R_values = tf.boolean_mask(Rating, mask)
    
    loss = tf.reduce_mean(tf.squared_difference(R_values,R_h_values))+beta*(tf.norm(Q)+tf.norm(P))
    

    optimizer = tf.train.GradientDescentOptimizer(lr)
    train_step = optimizer.minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    print("STARTING THE DESCENT")

    for i in range(epochs):
        if i % 10 == 0:
            l=sess.run(tf.reduce_mean(tf.squared_difference(R_h_values, R_values)),feed_dict={Rating:X})
            print("EPOCH " +str(i)+ ", Loss: " + str(l))
            if l<tolerance:
                break
        sess.run(train_step,feed_dict={Rating:X})
    
    final=tf.matmul(Q,P)
    final_res = sess.run([final])[0]
    sess.close()
    return np.round(final_res)
    

In [51]:
zz=Matrix_Factorization(R,epochs=150)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


C:\Users\Yamac\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:105: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 953518245 elements. This may consume a large amount of memory.
  num_elements)


STARTING THE DESCENT
EPOCH 0, Loss: 15.934969
EPOCH 10, Loss: 15.934969


KeyboardInterrupt: 

In [ ]:
tf.dynamic_partition(tf.reshape(r_hat, [-1]), user_ind * tf.shape(r_hat)[1] + item_ind, 1)

In [ ]:
zz=Matrix_Factorization(R_f,epochs=3)

In [ ]:
r_samp.shape

In [ ]:
user_ind=list(user_ind)
item_ind=list(item_ind)

In [ ]:
len(item_ind)

In [ ]:
tf.reset_default_graph() 

In [ ]:
Z=np.zeros((30000,60000))

In [ ]:
R.shape

In [ ]:
m,n=R_f.shape
    
Rating= tf.placeholder(tf.float32, [m,n])
Q = tf.Variable(tf.truncated_normal([int(m), 3], stddev=0.2, mean=1), name="users")
P=tf.Variable(tf.truncated_normal([3, int(n)], stddev=0.2, mean=1), name="beers")


r_hat=tf.matmul(Q,P)
mask =  tf.greater(Rating, 0)
R_h_values = tf.boolean_mask(r_hat, mask)
R_values = tf.boolean_mask(Rating, mask)
    
loss = tf.reduce_mean(tf.squared_difference(R_h_values, R_values))


In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
train_step = optimizer.minimize(loss)

In [ ]:
len(user_ind)

In [ ]:
X

In [ ]:
mask = tf.greater(X, 0)
non_zero_array = tf.boolean_mask(X, mask)

In [ ]:
Matrix_Factorization(X)

In [ ]:
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.GradientDescentOptimizer(0.0001)
train_step = optimizer.minimize(loss)


In [ ]:
R_values

In [ ]:
user_ind=user_ind.reshape(1571801,1)
item_ind=item_ind.reshape(1571801,1)

In [ ]:
list(user_ind)

In [ ]:
Rating.rank

In [ ]:
len(user_ind)

In [ ]:
Rating

In [ ]:
tf.gather(Rating,[user_ind])

In [ ]:

train_step = optimizer.minimize(loss)

In [ ]:
tf.Variable(tf.truncated_normal([5, 3], stddev=0.2, mean=0), name="users")

In [ ]:
Matrix_Factorization(R,epochs=2)

In [ ]:
X

In [ ]:


sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(10000):
    if i % 500 == 0:
        print("EPOCH " +str(i))
    sess.run(train_step,feed_dict={R:X})
    
final=tf.matmul(Q,P)
final_res = sess.run([final])


In [ ]:
R= tf.placeholder(tf.float32, [5,5])
Q = tf.Variable(tf.truncated_normal([5, 3], stddev=0.2, mean=0), name="users")
P=tf.Variable(tf.truncated_normal([3, 5], stddev=0.2, mean=0), name="beers")i

r_hat=tf.matmul(Q,P)

In [ ]:
R_h_values = tf.gather(tf.reshape(r_hat, [-1]), user_ind * tf.shape(r_hat)[1] + item_ind, name="existing_ratings2")
R_values=tf.gather(tf.reshape(R, [-1]), user_ind * tf.shape(r_hat)[1] + item_ind, name="existing_ratings")

In [ ]:
loss = tf.reduce_mean(tf.squared_difference(R_h_values, R_values))
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.GradientDescentOptimizer(0.002)
train_step = optimizer.minimize(loss, global_step=global_step)


In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())


In [ ]:
for i in range(10000):
    if i % 500 == 0:
        print("EPOCH " +str(i))
    sess.run(train_step,feed_dict={R:X})
    
final=tf.matmul(Q,P)
final_res = sess.run([final])

In [ ]:
final_res

In [ ]:
Z=np.vstack((X,X))

In [ ]:
n,m=R.shape
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for k in range(len(i)):
    sess.run(grad_descent)
        
            
            

In [ ]:
int(Q[0,1])

In [ ]:
eis=R[0,0]-tf.tensordot(Q[0],P[:,0],1)

In [ ]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
out=sess.run(rows,feed_dict={r:X})
print(out)

In [ ]:
yy=np.where(X!=0)

In [ ]:
np.sum(X!=0)

In [ ]:
def p_q(k):
    ps=[]
    qs=[]
    for i in range(k):
        ps.append(tf.Variable(tf.zeros([k])))
        qs.append(tf.Variable(tf.zeros([k])))
    return qs,ps

In [ ]:
sA = sparse.csr_matrix(R)

In [ ]:
sA[0,5]

In [ ]:
R

In [ ]:
count=0
for i in R:
    if np.sum(i>0)<=3:
        count+=1
count

In [ ]:
z=np.where(R>0)

In [ ]:
len(z[0])

In [ ]:
k=z[0]
j=z[1]

In [ ]:
Q=np.random.rand(6,3)
P=np.random.rand(6,3).T

In [ ]:
for i in range (1571801):
    

In [ ]:
Z=sparse.rand(3, 4, density=0.25, format="csr", random_state=42)

In [ ]:
Z.todense()

### EXPERIMENTAL

In [ ]:
def matrix_factor(X,epochs=15000,k=4,lr=0.007,beta=0.001):
    n,m=X.shape
    Q=np.random.rand(n,k)
    P=np.random.rand(m,k).T  
    z_i,z_j=np.where(X>0)
    z_i=list(z_i)
    z_j=list(z_j)
    n=len(z_i)
    for epoch in range(epochs):
        print("Epoch : " + str(epoch))
        for u in range(n):
            i=z_i[u]
            j=z_j[u]
            Q[i]=Q[i]+2*lr*(X[i,j]-Q[i]@P[:,j])*P[:,j]-beta*Q[i]
            P[:,j]=P[:,j]+2*lr*(X[i,j]-Q[i]@P[:,j])*Q[i]-beta*P[:,j]
    return Q@P
     

    
    

In [ ]:
R= tf.placeholder(tf.float32, [5,5])
Q = tf.Variable(tf.truncated_normal([5, 3], stddev=0.2, mean=0), name="users")
P=tf.Variable(tf.truncated_normal([3, 5], stddev=0.2, mean=0), name="beers")

In [ ]:
P[1]-0.002*Q[:,0]

In [ ]:
Q[:,0]+2.0*0.0002*(X[0,1]-tf.tensordot(Q[:,1],P[0],axes=1))*P[1]-0.002*Q[:,0]

In [ ]:
i=0
j=1
lr=0.002
beta=0.001

In [ ]:
Q[:,i]=Q[:,i]+2.0*lr*(X[i,j]-tf.tensordot(Q[:,1],P[0],axes=1))*P[j]-beta*Q[:,i]

In [ ]:
Q[i]=Q[i]+2*lr*(X[i,j]-tf.tensordot(Q[:,1],P[0],axes=1))*P[j]-beta*Q[:,i]

In [ ]:
def grad_descent(X,Q,P,i,j):
    lr=0.002
    beta=0.001
    Q[i]=Q[i]+2*lr*(X[i,j]-tf.matmul(Q[i],P[:,j]))*P[:,j]-beta*Q[i]
    P[:,j]=P[:,j]+2*lr*(X[i,j]-tf.matmul(Q[i],P[:,j]))*Q[i]-beta*P[:,j]
    Q@P

In [ ]:
Q=np.random.rand(6,3)
P=np.random.rand(6,3).T

In [ ]:
grad_descent(X,Q,P,0,1)

In [ ]:
np.save("den.np",R_f)